In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#geospatial imports
import geopandas as gpd
import shapely
from fiona.crs import from_epsg
from shapely.geometry import LineString, Point
from geopandas.tools import sjoin
#regression
import statsmodels.formula.api as smf
from functools import reduce

# Purpose of this notebook

To perform analysis 

# Reading 311 data

In [2]:
complaints = pd.read_csv('2018_dep_311.csv')
len(complaints)

204709

In [3]:
complaints2 = complaints[complaints['Complaint Type']=='Noise']
len(complaints2)

60699

In [4]:
complaints2['complaintDate'] = pd.to_datetime(complaints2['Created Date'], errors = 'coerce')
complaints2['resolveDate'] = pd.to_datetime(complaints2['Closed Date'], errors = 'coerce')
complaints2['monthnumber'] = complaints2['complaintDate'].apply(lambda x: x.month)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to 

Geographical transformations

In [5]:
complaints2['lonlat']=list(zip(complaints2.Longitude,complaints2.Latitude))
complaints2 = gpd.GeoDataFrame(complaints2)
complaints2.crs = from_epsg(4326)
complaints2['geometry'] = complaints2[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))
complaints2.head()


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location,complaintDate,resolveDate,monthnumber,lonlat,geometry
2,40289117,09/15/2018 10:09:00 AM,09/15/2018 11:00:00 AM,DEP,Department of Environmental Protection,Noise,Noise: Alarms (NR3),NaN,11366.0,158-37 78 AVENUE,...,NaN,NaN,40.722390,-73.809519,"(40.72238976571024, -73.80951881866362)",2018-09-15 10:09:00,2018-09-15 11:00:00,9,"(-73.8095188187, 40.7223897657)",POINT (-73.80951881866362 40.72238976571025)
16,39351871,06/02/2018 01:07:00 PM,06/02/2018 02:40:00 PM,DEP,Department of Environmental Protection,Noise,"Noise, Ice Cream Truck (NR4)",NaN,10463.0,NaN,...,NaN,NaN,40.884868,-73.890804,"(40.88486808373441, -73.89080395748026)",2018-06-02 13:07:00,2018-06-02 14:40:00,6,"(-73.8908039575, 40.8848680837)",POINT (-73.89080395748027 40.88486808373441)
17,38730897,03/19/2018 12:09:00 PM,04/22/2018 10:00:00 AM,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),NaN,11225.0,1016 CARROLL STREET,...,NaN,NaN,40.667710,-73.957421,"(40.66770996768325, -73.95742120985392)",2018-03-19 12:09:00,2018-04-22 10:00:00,3,"(-73.9574212099, 40.6677099677)",POINT (-73.95742120985392 40.66770996768325)
19,39353719,06/02/2018 07:21:00 AM,06/02/2018 09:00:00 AM,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),NaN,11222.0,5 COMMERCIAL STREET,...,NaN,NaN,40.735622,-73.959698,"(40.735621785012405, -73.95969800191561)",2018-06-02 07:21:00,2018-06-02 09:00:00,6,"(-73.9596980019, 40.735621785)",POINT (-73.95969800191561 40.73562178501241)
20,39356684,06/02/2018 08:39:00 AM,06/09/2018 02:00:00 PM,DEP,Department of Environmental Protection,Noise,Noise: Construction Before/After Hours (NM1),NaN,10013.0,250 WEST STREET,...,NaN,NaN,40.721704,-74.011317,"(40.721704134391594, -74.01131712436428)",2018-06-02 08:39:00,2018-06-09 14:00:00,6,"(-74.0113171244, 40.7217041344)",POINT (-74.01131712436428 40.7217041343916)


# Reading permits

In [6]:
permits = pd.read_csv('permit5.csv')
permits = permits.drop(['Unnamed: 0', 'index'], axis=1)
permits['JobStartDate'] = pd.to_datetime(permits['JobStartDate'], errors = 'coerce')
permits['monthnumber'] = permits['JobStartDate'].apply(lambda x: x.month)

Geographical transformation

In [7]:
permits['lonlat']=list(zip(permits.LONGITUDE,permits.LATITUDE))
permits['geometry'] = permits[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

In [8]:
permits.head()

,﻿BOROUGH,CENSUS_TRACT,NTA_NAME,Zip Code,Block,Lot,LATITUDE,LONGITUDE,Job Type,Work Type,...,Job Start Date,JobStartDate,ExpireDate,JobYear,IssueDate,ExpireYear,IssueYear,monthnumber,lonlat,geometry
0,BROOKLYN,85.0,Carroll Gardens-Columbia Street-Red Hook,11231.0,538.0,1.0,40.675823,-74.006850,A2,OT,...,07/02/2018 12:00:00 AM,2018-07-02,2020-02-27,2018.0,2019-03-01,2020.0,2019.0,7.0,"(-74.00685, 40.675823)",POINT (-74.00685 40.675823)
1,BROOKLYN,85.0,Carroll Gardens-Columbia Street-Red Hook,11231.0,538.0,1.0,40.675823,-74.006850,A2,OT,...,07/02/2018 12:00:00 AM,2018-07-02,2020-02-27,2018.0,2019-03-01,2020.0,2019.0,7.0,"(-74.00685, 40.675823)",POINT (-74.00685 40.675823)
2,BROOKLYN,85.0,Carroll Gardens-Columbia Street-Red Hook,11231.0,538.0,1.0,40.675823,-74.006850,A2,OT,...,07/02/2018 12:00:00 AM,2018-07-02,2020-02-27,2018.0,2019-03-01,2020.0,2019.0,7.0,"(-74.00685, 40.675823)",POINT (-74.00685 40.675823)
3,BROOKLYN,349.0,Crown Heights North,11213.0,1397.0,6.0,40.668765,-73.930659,A2,OT,...,03/16/2018 12:00:00 AM,2018-03-16,2020-02-03,2018.0,2019-03-01,2020.0,2019.0,3.0,"(-73.930659, 40.668765)",POINT (-73.93065899999998 40.668765)
4,BROOKLYN,571.0,Greenpoint,11222.0,2684.0,2.0,40.725028,-73.946962,A2,OT,...,02/08/2018 12:00:00 AM,2018-02-08,2020-02-29,2018.0,2019-03-01,2020.0,2019.0,2.0,"(-73.946962, 40.725028)",POINT (-73.946962 40.725028)


# Combined analysis

All construction in NYC and complaints in NYC, 2018

In [9]:
allmonths_311 = pd.DataFrame(permits.groupby([ 'monthnumber']).size()).reset_index()
allmonths_311.columns = ['Month','count']
allmonths_311.head()

,Month,count
0,1.0,10578
1,2.0,9274
2,3.0,11300
3,4.0,10875
4,5.0,11941


In [10]:
allmonths_permits = pd.DataFrame(complaints2.groupby([ 'monthnumber']).size()).reset_index()
allmonths_permits.columns = ['Month', 'count']
allmonths_permits.head()


,Month,count
0,1,3934
1,2,4413
2,3,5031
3,4,6186
4,5,6234


In [11]:
allmonths = pd.merge(allmonths_311, allmonths_permits, on='Month')
allmonths.columns = ['Month', 'complaints', 'dob_permit']
allmonths

,Month,complaints,dob_permit
0,1.0,10578,3934
1,2.0,9274,4413
2,3.0,11300,5031
3,4.0,10875,6186
4,5.0,11941,6234
5,6.0,10307,6085
6,7.0,11158,4324
7,8.0,11500,3991
8,9.0,9112,4992
9,10.0,10433,6196


In [12]:
dob_permits = allmonths['dob_permit']
complaints_311 = allmonths['complaints']
mod = smf.ols(formula='complaints_311 ~ dob_permits', data = allmonths).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:         complaints_311   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.8534
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.377
Time:                        11:39:58   Log-Likelihood:                -101.84
No. Observations:                  12   AIC:                             207.7
Df Residuals:                      10   BIC:                             208.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept    8271.9918   2184.149      3.787      

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


# Boroughs

In [13]:
brooklyn_permits = permits[permits['﻿BOROUGH']=='BROOKLYN']
manhattan_permits = permits[permits['﻿BOROUGH']=='MANHATTAN']
queens_permits = permits[permits['﻿BOROUGH']=='QUEENS']
bronx_permits = permits[permits['﻿BOROUGH']=='BRONX']
staten_permits = permits[permits['﻿BOROUGH']=='STATEN ISLAND']

In [14]:
complaints_boroughs = pd.DataFrame(complaints2.groupby(['Borough', 'monthnumber']).size()).reset_index()
complaints_boroughs.columns = ['Boroughs','month', 'count']
complaints_boroughs.head()


,Boroughs,month,count
0,BRONX,1,190
1,BRONX,2,186
2,BRONX,3,237
3,BRONX,4,388
4,BRONX,5,330


## Brooklyn

Overall

In [15]:
brooklyn_dob = pd.DataFrame(brooklyn_permits.groupby([ 'monthnumber']).size()).reset_index()
brooklyn_dob.columns = ['month','count']
brooklyn_dob 


,month,count
0,1.0,2695
1,2.0,2550
2,3.0,2924
3,4.0,2832
4,5.0,3204
5,6.0,2561
6,7.0,2772
7,8.0,2795
8,9.0,2169
9,10.0,2536


In [16]:
brooklyn_complaints = complaints_boroughs[complaints_boroughs['Boroughs'] == 'BROOKLYN']
brooklyn_complaints.drop(['Boroughs'], axis=1, inplace=True)
brooklyn_complaints

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,month,count
12,1,1158
13,2,1158
14,3,1365
15,4,1548
16,5,1633
17,6,1523
18,7,1185
19,8,1065
20,9,1322
21,10,1596


In [17]:
brooklyn = pd.merge(brooklyn_complaints, brooklyn_dob, on='month')
brooklyn.columns = ['Month', 'complaints', 'dob_permit']
brooklyn

,Month,complaints,dob_permit
0,1,1158,2695
1,2,1158,2550
2,3,1365,2924
3,4,1548,2832
4,5,1633,3204
5,6,1523,2561
6,7,1185,2772
7,8,1065,2795
8,9,1322,2169
9,10,1596,2536


In [18]:
bk_permits = brooklyn['dob_permit']
complaints_bk = brooklyn['complaints']
mod = smf.ols(formula='complaints_bk ~ bk_permits', data = brooklyn).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:          complaints_bk   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     1.577
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.238
Time:                        11:39:58   Log-Likelihood:                -79.302
No. Observations:                  12   AIC:                             162.6
Df Residuals:                      10   BIC:                             163.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    759.7185    459.039      1.655      0.1

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


Job Types

In [19]:
brooklyn_permits.columns

Index(['﻿BOROUGH', 'CENSUS_TRACT', 'NTA_NAME', 'Zip Code', 'Block', 'Lot',
       'LATITUDE', 'LONGITUDE', 'Job Type', 'Work Type', 'Permit Type',
       'Permit Subtype', 'Oil Gas', 'Site Fill', 'Issuance Date',
       'Expiration Date', 'Job Start Date', 'JobStartDate', 'ExpireDate',
       'JobYear', 'IssueDate', 'ExpireYear', 'IssueYear', 'monthnumber',
       'lonlat', 'geometry'],
      dtype='object')

In [20]:
brooklyn_permits['Work Type'].unique()

array(['OT', nan, 'PL', 'SP', 'BL', 'EQ', 'MH', 'FP', 'SD', 'FB', 'FS',
       'CC'], dtype=object)

In [21]:
brooklyn_permits['Permit Type'].unique()

array(['EW', 'FO', 'PL', 'NB', 'EQ', 'AL', 'DM', 'SG'], dtype=object)

In [22]:
brooklyn_job = pd.DataFrame(brooklyn_permits.groupby(['Job Type', 'monthnumber']).size()).reset_index()
brooklyn_job.columns = ['Job Type', 'month', 'count']

brooklyn_a1 = brooklyn_job[brooklyn_job['Job Type'] == 'A1']
brooklyn_a2 = brooklyn_job[brooklyn_job['Job Type'] == 'A2']
brooklyn_a3 = brooklyn_job[brooklyn_job['Job Type'] == 'A3']
brooklyn_dm = brooklyn_job[brooklyn_job['Job Type'] == 'DM']
brooklyn_nb = brooklyn_job[brooklyn_job['Job Type'] == 'NB']
brooklyn_sg = brooklyn_job[brooklyn_job['Job Type'] == 'SG']

brooklyn_alljobs = [brooklyn_a1, brooklyn_a2, brooklyn_a3, brooklyn_dm, brooklyn_nb, brooklyn_sg]


In [23]:
#code credit: https://stackoverflow.com/questions/44327999/python-pandas-merge-multiple-dataframes/44338256

brooklyn_jobtypes = reduce(lambda left,right: pd.merge(left,right,on=['month'],how='outer'
                                            ), brooklyn_alljobs)

brooklyn_jobtypes.columns = ['A1', 'month', 'A1_count', 
                             'A2', 'A2_count', 
                             'A3', 'A3_count',
                            'DM', 'DM_count',
                            'NB', 'NB_count',
                            'SG', 'SG_count']

brooklyn_jobtypes.drop(['A1','A2','A3','DM','NB','SG'], axis=1, inplace=True)

brooklyn_jobtypes2 = pd.merge(brooklyn_complaints, brooklyn_jobtypes, on='month')

brooklyn_jobtypes2.rename(columns={'count':'complaints_bk'}, inplace=True) 
brooklyn_jobtypes2


,month,complaints_bk,A1_count,A2_count,A3_count,DM_count,NB_count,SG_count
0,1,1158,381,1904,116,61,229,4.0
1,2,1158,365,1607,104,94,373,7.0
2,3,1365,398,1964,116,68,375,3.0
3,4,1548,345,1982,101,73,319,12.0
4,5,1633,423,2166,131,72,407,5.0
5,6,1523,307,1828,103,66,252,5.0
6,7,1185,343,2013,133,72,211,NaN
7,8,1065,376,1978,127,65,249,NaN
8,9,1322,260,1619,87,73,130,NaN
9,10,1596,289,1824,128,94,201,NaN


In [24]:
dob_permits = brooklyn_jobtypes2[['A1_count', 'A2_count', 'A3_count', 'DM_count','NB_count']]
# complaints_311 = allmonths['complaints']
mod = smf.ols(formula='complaints_bk ~ dob_permits', data = brooklyn_jobtypes2).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:          complaints_bk   R-squared:                       0.768
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     3.974
Date:                Fri, 12 Apr 2019   Prob (F-statistic):             0.0615
Time:                        11:39:58   Log-Likelihood:                -71.414
No. Observations:                  12   AIC:                             154.8
Df Residuals:                       6   BIC:                             157.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept        278.6453    533.411      0.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


## Manhattan

In [25]:
man_dob = pd.DataFrame(manhattan_permits.groupby([ 'monthnumber']).size()).reset_index()
man_dob.columns = ['month','count']
man_dob 

,month,count
0,1.0,4416
1,2.0,3830
2,3.0,4813
3,4.0,4574
4,5.0,4893
5,6.0,4504
6,7.0,4730
7,8.0,4860
8,9.0,3816
9,10.0,4165


In [26]:
man_complaints = complaints_boroughs[complaints_boroughs['Boroughs'] == 'MANHATTAN']
man_complaints.drop(['Boroughs'], axis=1, inplace=True)
man_complaints

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,month,count
24,1,1938
25,2,2321
26,3,2546
27,4,3192
28,5,2928
29,6,2573
30,7,1689
31,8,1677
32,9,2184
33,10,3034


In [27]:
man = pd.merge(man_complaints, man_dob, on='month')
man.columns = ['Month', 'complaints', 'dob_permit']
man

,Month,complaints,dob_permit
0,1,1938,4416
1,2,2321,3830
2,3,2546,4813
3,4,3192,4574
4,5,2928,4893
5,6,2573,4504
6,7,1689,4730
7,8,1677,4860
8,9,2184,3816
9,10,3034,4165


In [28]:
man_permits = man['dob_permit']
complaints_man = man['complaints']
mod = smf.ols(formula='complaints_man ~ man_permits', data = man).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:         complaints_man   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.063
Method:                 Least Squares   F-statistic:                    0.3488
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.568
Time:                        11:39:58   Log-Likelihood:                -91.397
No. Observations:                  12   AIC:                             186.8
Df Residuals:                      10   BIC:                             187.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept    1675.8337   1186.881      1.412      

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


Job types

In [29]:
man_job = pd.DataFrame(manhattan_permits.groupby(['Job Type', 'monthnumber']).size()).reset_index()
man_job.columns = ['Job Type', 'month', 'count']

man_a1 = man_job[man_job['Job Type'] == 'A1']
man_a2 = man_job[man_job['Job Type'] == 'A2']
man_a3 = man_job[man_job['Job Type'] == 'A3']
man_dm = man_job[man_job['Job Type'] == 'DM']
man_nb = man_job[man_job['Job Type'] == 'NB']
man_sg = man_job[man_job['Job Type'] == 'SG']

man_alljobs = [man_a1, man_a2, man_a3, man_dm, man_nb, man_sg]

In [30]:
man_jobtypes = reduce(lambda left,right: pd.merge(left,right,on=['month'],how='outer'
                                            ), man_alljobs)

man_jobtypes.columns = ['A1', 'month', 'A1_count', 
                             'A2', 'A2_count', 
                             'A3', 'A3_count',
                            'DM', 'DM_count',
                            'NB', 'NB_count',
                            'SG', 'SG_count']

man_jobtypes.drop(['A1','A2','A3','DM','NB','SG'], axis=1, inplace=True)

man_jobtypes2 = pd.merge(man_complaints, man_jobtypes, on='month')

man_jobtypes2.rename(columns={'count':'complaints_man'}, inplace=True) 
man_jobtypes2

,month,complaints_man,A1_count,A2_count,A3_count,DM_count,NB_count,SG_count
0,1,1938,197,3903,185,25,103,3.0
1,2,2321,168,3396,150,15,96,5.0
2,3,2546,111,4441,150,18,86,7.0
3,4,3192,141,4224,141,15,50,3.0
4,5,2928,123,4458,214,20,73,5.0
5,6,2573,124,4125,172,14,65,4.0
6,7,1689,131,4367,151,18,59,4.0
7,8,1677,114,4516,169,22,39,NaN
8,9,2184,102,3492,167,14,41,NaN
9,10,3034,86,3816,166,41,50,6.0


In [31]:
dob_permits = man_jobtypes2[['A1_count', 'A2_count', 'A3_count', 'DM_count','NB_count']]
mod = smf.ols(formula='complaints_man ~ dob_permits', data = man_jobtypes2).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:         complaints_man   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                 -0.581
Method:                 Least Squares   F-statistic:                    0.1919
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.955
Time:                        11:39:58   Log-Likelihood:                -90.713
No. Observations:                  12   AIC:                             193.4
Df Residuals:                       6   BIC:                             196.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept       1550.6492   1607.217      0.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


## Bronx

In [32]:
bronx_dob = pd.DataFrame(bronx_permits.groupby([ 'monthnumber']).size()).reset_index()
bronx_dob.columns = ['month','count']

bronx_complaints = complaints_boroughs[complaints_boroughs['Boroughs'] == 'BRONX']
bronx_complaints.drop(['Boroughs'], axis=1, inplace=True)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
bronx = pd.merge(bronx_complaints, bronx_dob, on='month')
bronx.columns = ['Month', 'complaints', 'dob_permit']
bronx

,Month,complaints,dob_permit
0,1,190,791
1,2,186,621
2,3,237,748
3,4,388,654
4,5,330,780
5,6,373,700
6,7,242,862
7,8,278,817
8,9,284,668
9,10,300,1025


In [34]:
permits_bronx = bronx['dob_permit']
complaints_bronx = bronx['complaints']
mod = smf.ols(formula='complaints_bronx ~ permits_bronx', data = bronx).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:       complaints_bronx   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.097
Method:                 Least Squares   F-statistic:                   0.02506
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.877
Time:                        11:39:58   Log-Likelihood:                -67.087
No. Observations:                  12   AIC:                             138.2
Df Residuals:                      10   BIC:                             139.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept       249.0593    133.852      1.861

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


Job Types

In [35]:
bronx_job = pd.DataFrame(bronx_permits.groupby(['Job Type', 'monthnumber']).size()).reset_index()
bronx_job.columns = ['Job Type', 'month', 'count']

bronx_a1 = bronx_job[bronx_job['Job Type'] == 'A1']
bronx_a2 = bronx_job[bronx_job['Job Type'] == 'A2']
bronx_a3 = bronx_job[bronx_job['Job Type'] == 'A3']
bronx_dm = bronx_job[bronx_job['Job Type'] == 'DM']
bronx_nb = bronx_job[bronx_job['Job Type'] == 'NB']
bronx_sg = bronx_job[bronx_job['Job Type'] == 'SG']

bronx_alljobs = [bronx_a1, bronx_a2, bronx_a3, bronx_dm, bronx_nb, bronx_sg]


In [42]:
bronx_jobtypes = reduce(lambda left,right: pd.merge(left,right,on=['month'],how='outer'
                                            ), bronx_alljobs)

bronx_jobtypes.columns = ['A1', 'month', 'A1_count', 
                             'A2', 'A2_count', 
                             'A3', 'A3_count',
                            'DM', 'DM_count',
                            'NB', 'NB_count',
                            'SG', 'SG_count']

bronx_jobtypes.drop(['A1','A2','A3','DM','NB','SG'], axis=1, inplace=True)

bronx_jobtypes2 = pd.merge(bronx_complaints, bronx_jobtypes, on='month')

bronx_jobtypes2.rename(columns={'count':'complaints_bronx'}, inplace=True) 
bronx_jobtypes2

,month,complaints_bronx,A1_count,A2_count,A3_count,DM_count,NB_count,SG_count
0,1,190,45,494,36,26,186,4.0
1,2,186,40,417,51,15,98,NaN
2,3,237,66,524,61,28,69,NaN
3,4,388,43,492,41,14,62,2.0
4,5,330,32,626,50,17,46,9.0
5,6,373,47,517,53,12,71,NaN
6,7,242,38,695,57,24,48,NaN
7,8,278,54,616,49,41,57,NaN
8,9,284,33,501,56,28,50,NaN
9,10,300,43,790,80,41,71,NaN


In [44]:
dob_permits = bronx_jobtypes2[['A1_count', 'A2_count', 'A3_count', 'DM_count','NB_count']]
mod = smf.ols(formula='complaints_bronx ~ dob_permits', data = bronx_jobtypes2).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:       complaints_bronx   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                 -0.157
Method:                 Least Squares   F-statistic:                    0.7020
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.643
Time:                        11:41:34   Log-Likelihood:                -64.339
No. Observations:                  12   AIC:                             140.7
Df Residuals:                       6   BIC:                             143.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept        215.5432    152.309      1.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


## Queens

In [36]:
queens_dob = pd.DataFrame(queens_permits.groupby([ 'monthnumber']).size()).reset_index()
queens_dob.columns = ['month','count']

queens_complaints = complaints_boroughs[complaints_boroughs['Boroughs'] == 'QUEENS']
queens_complaints.drop(['Boroughs'], axis=1, inplace=True)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
queens = pd.merge(queens_complaints, queens_dob, on='month')
queens.columns = ['Month', 'complaints', 'dob_permit']
queens

,Month,complaints,dob_permit
0,1,552,2198
1,2,671,1858
2,3,773,2353
3,4,869,2288
4,5,1141,2487
5,6,1445,2076
6,7,1064,2303
7,8,847,2393
8,9,1060,1932
9,10,1112,2151


In [38]:
permits_queens = queens['dob_permit']
complaints_queens = queens['complaints']
mod = smf.ols(formula='complaints_queens ~ permits_queens', data = queens).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:      complaints_queens   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.095
Method:                 Least Squares   F-statistic:                   0.04814
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.831
Time:                        11:39:58   Log-Likelihood:                -82.126
No. Observations:                  12   AIC:                             168.3
Df Residuals:                      10   BIC:                             169.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept        810.3624    621.227      1.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


Job Types

In [45]:
queens_job = pd.DataFrame(queens_permits.groupby(['Job Type', 'monthnumber']).size()).reset_index()
queens_job.columns = ['Job Type', 'month', 'count']

queens_a1 = queens_job[queens_job['Job Type'] == 'A1']
queens_a2 = queens_job[queens_job['Job Type'] == 'A2']
queens_a3 = queens_job[queens_job['Job Type'] == 'A3']
queens_dm = queens_job[queens_job['Job Type'] == 'DM']
queens_nb = queens_job[queens_job['Job Type'] == 'NB']
queens_sg = queens_job[queens_job['Job Type'] == 'SG']

queens_alljobs = [queens_a1, queens_a2, queens_a3, queens_dm, queens_nb, queens_sg]


In [46]:
queens_jobtypes = reduce(lambda left,right: pd.merge(left,right,on=['month'],how='outer'
                                            ), queens_alljobs)

queens_jobtypes.columns = ['A1', 'month', 'A1_count', 
                             'A2', 'A2_count', 
                             'A3', 'A3_count',
                            'DM', 'DM_count',
                            'NB', 'NB_count',
                            'SG', 'SG_count']

queens_jobtypes.drop(['A1','A2','A3','DM','NB','SG'], axis=1, inplace=True)

queens_jobtypes2 = pd.merge(queens_complaints, queens_jobtypes, on='month')

queens_jobtypes2.rename(columns={'count':'complaints_bronx'}, inplace=True) 
queens_jobtypes2

,month,complaints_bronx,A1_count,A2_count,A3_count,DM_count,NB_count,SG_count
0,1,552,285,1461,72,53,322,5.0
1,2,671,273,1188,48,76,272,1.0
2,3,773,330,1589,86,63,279,6.0
3,4,869,303,1500,70,53,352,10.0
4,5,1141,436,1572,85,73,304,17.0
5,6,1445,308,1379,75,73,241,NaN
6,7,1064,396,1546,80,64,217,NaN
7,8,847,299,1538,89,65,401,1.0
8,9,1060,244,1297,79,52,260,NaN
9,10,1112,274,1442,91,60,280,4.0


In [47]:
dob_permits = queens_jobtypes2[['A1_count', 'A2_count', 'A3_count', 'DM_count','NB_count']]
mod = smf.ols(formula='complaints_queens ~ dob_permits', data = bronx_jobtypes2).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:      complaints_queens   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                 -0.167
Method:                 Least Squares   F-statistic:                    0.6852
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.652
Time:                        11:44:54   Log-Likelihood:                -79.444
No. Observations:                  12   AIC:                             170.9
Df Residuals:                       6   BIC:                             173.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         86.3009   1393.568      0.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


# Staten island

In [39]:
staten_dob = pd.DataFrame(staten_permits.groupby([ 'monthnumber']).size()).reset_index()
staten_dob.columns = ['month','count']

staten_complaints = complaints_boroughs[complaints_boroughs['Boroughs'] == 'STATEN ISLAND']
staten_complaints.drop(['Boroughs'], axis=1, inplace=True)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
staten = pd.merge(staten_complaints, staten_dob, on='month')
staten.columns = ['Month', 'complaints', 'dob_permit']
staten

,Month,complaints,dob_permit
0,1,96,478
1,2,77,415
2,3,110,462
3,4,189,527
4,5,202,577
5,6,171,466
6,7,144,491
7,8,124,635
8,9,142,527
9,10,154,556


In [41]:
permits_staten = staten['dob_permit']
complaints_staten = staten['complaints']
mod = smf.ols(formula='complaints_staten ~ permits_staten', data = staten).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:      complaints_staten   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     1.711
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.220
Time:                        11:39:59   Log-Likelihood:                -58.688
No. Observations:                  12   AIC:                             121.4
Df Residuals:                      10   BIC:                             122.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept         48.2851     70.784      0.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))


Job Types

In [48]:
staten_job = pd.DataFrame(staten_permits.groupby(['Job Type', 'monthnumber']).size()).reset_index()
staten_job.columns = ['Job Type', 'month', 'count']

staten_a1 = staten_job[staten_job['Job Type'] == 'A1']
staten_a2 = staten_job[staten_job['Job Type'] == 'A2']
staten_a3 = staten_job[staten_job['Job Type'] == 'A3']
staten_dm = staten_job[staten_job['Job Type'] == 'DM']
staten_nb = staten_job[staten_job['Job Type'] == 'NB']
staten_sg = staten_job[staten_job['Job Type'] == 'SG']

staten_alljobs = [staten_a1, staten_a2, staten_a3, staten_dm, staten_nb, staten_sg]


In [49]:
staten_jobtypes = reduce(lambda left,right: pd.merge(left,right,on=['month'],how='outer'
                                            ), staten_alljobs)

staten_jobtypes.columns = ['A1', 'month', 'A1_count', 
                             'A2', 'A2_count', 
                             'A3', 'A3_count',
                            'DM', 'DM_count',
                            'NB', 'NB_count',
                            'SG', 'SG_count']

staten_jobtypes.drop(['A1','A2','A3','DM','NB','SG'], axis=1, inplace=True)

staten_jobtypes2 = pd.merge(staten_complaints, staten_jobtypes, on='month')

staten_jobtypes2.rename(columns={'count':'complaints_bronx'}, inplace=True) 
staten_jobtypes2

,month,complaints_bronx,A1_count,A2_count,A3_count,DM_count,NB_count,SG_count
0,1,96,35,260,4,20,155,4.0
1,2,77,21,229,7,40,118,NaN
2,3,110,19,300,19,23,101,NaN
3,4,189,50,326,10,28,113,NaN
4,5,202,38,351,11,24,153,NaN
5,6,171,44,235,10,20,157,NaN
6,7,144,38,312,11,34,96,NaN
7,8,124,27,420,15,31,142,NaN
8,9,142,32,349,4,21,121,NaN
9,10,154,42,335,14,15,150,NaN


In [50]:
dob_permits = staten_jobtypes2[['A1_count', 'A2_count', 'A3_count', 'DM_count','NB_count']]
mod = smf.ols(formula='complaints_staten ~ dob_permits', data = bronx_jobtypes2).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:      complaints_staten   R-squared:                       0.700
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     2.805
Date:                Fri, 12 Apr 2019   Prob (F-statistic):              0.121
Time:                        11:47:57   Log-Likelihood:                -52.405
No. Observations:                  12   AIC:                             116.8
Df Residuals:                       6   BIC:                             119.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Intercept        -21.6661     71.850     -0.

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/scipy/stats/stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  "anyway, n=%i" % int(n))
